In [10]:
import json
from glob import glob
from tqdm import tqdm

In [11]:
files = glob('answer-factually-wrong/*.json')
len(files)

1139290

In [23]:
mapping = {}
with open('mixtral-factually-wrong-code.translated') as fopen:
    r = json.load(fopen)
    for r_ in r:
        mapping[r_['src']] = r_['r']

In [24]:
with open('factually-wrong-qa-coding.jsonl', 'w') as fl:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        if '```' not in data[1]:
            continue
        d = {
            'doc': data[0][0],
            'question': data[0][1],
            'answer': data[1].strip()
        }
        d['question_ms'] = mapping.get(d['question'])
        d['answer_ms'] = mapping.get(d['answer'])
        fl.write(f'{json.dumps(d)}\n')
        fl.flush()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1139290/1139290 [06:06<00:00, 3106.09it/s]


In [26]:
!tail -n 3 factually-wrong-qa-coding.jsonl

{"doc": "integer y_as_integer = int(y) print(y_as_integer) # Output: 10 # Converting string to float y_as_float = float(y) print(y_as_float) # Output: 10.0 ``` In the example above, we demonstrate type conversion in Python. We convert an integer to a string using the `str()` function, a string to an integer using the `int()` function, and a string to a float using the `float()` function. Type conversion is useful when you need to perform operations or comparisons between different data types. It allows you to ensure that the data types are compatible and can be used together. ```python # Example of type conversion in Python x = 5 y = \"10\" # Converting integer to string x_as_string = str(x) print(x_as_string) # Output: \"5\" # Converting string to integer y_as_integer = int(y) print(y_as_integer) # Output: 10 # Converting string to float y_as_float = float(y) print(y_as_float) # Output: 10.0 ``` In this example, we perform type conversion between different data types in Python. We con

In [25]:
import re

mapping = {}
for f in glob('texts.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
            
len(mapping)

2278428

In [29]:
with open('factually-wrong-qa.jsonl', 'w') as fl:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        if '```' in data[1].strip():
            continue
        d = {
            'doc': data[0][0],
            'question': data[0][1],
            'answer': data[1].strip()
        }
        d['question_ms'] = mapping.get(d['question'])
        d['answer_ms'] = mapping.get(d['answer'])
        fl.write(f'{json.dumps(d)}\n')
        fl.flush()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1139290/1139290 [00:49<00:00, 23063.77it/s]


In [30]:
!tail -n 3 factually-wrong-qa.jsonl

{"doc": "to pack and unpack data. 10.4. Bitwise Encryption Bitwise encryption is a technique that allows you to encrypt data using bitwise operations. This technique leverages the properties of bitwise XOR (`^`) to perform the encryption and decryption operations. Bitwise encryption can be used to protect sensitive data, such as passwords or private keys, from unauthorized access. By applying a bitwise XOR operation with a secret key, the data can be encrypted in a reversible manner. Bitwise encryption techniques often involve using a secret key that is known only to the sender and receiver. The key is used to perform the bitwise XOR operation on the data, producing the encrypted result. To decrypt the data, the same key is used to perform the bitwise XOR operation again, recovering the original data. Bitwise encryption is a simple form of encryption and may not provide strong security against advanced attacks. It is often used in combination with other encryption techniques to provide

In [31]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='factually-wrong-qa-coding.jsonl',
    path_in_repo='factually-wrong-qa-coding.jsonl',
    repo_id='mesolitica/factually-wrong-QA',
    repo_type='dataset',
)

factually-wrong-qa-coding.jsonl:   0%|          | 0.00/446M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/mixtral-factually-wrong-QA/commit/3e0a7cb0d4fd83829d9a248dbab43085111c8c2f', commit_message='Upload factually-wrong-qa-coding.jsonl with huggingface_hub', commit_description='', oid='3e0a7cb0d4fd83829d9a248dbab43085111c8c2f', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
api.upload_file(
    path_or_fileobj='factually-wrong-qa.jsonl',
    path_in_repo='factually-wrong-qa.jsonl',
    repo_id='mesolitica/factually-wrong-QA',
    repo_type='dataset',
)

factually-wrong-qa.jsonl:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/mixtral-factually-wrong-QA/commit/560f4d7d00954a4573721ff06c233583c32a798a', commit_message='Upload factually-wrong-qa.jsonl with huggingface_hub', commit_description='', oid='560f4d7d00954a4573721ff06c233583c32a798a', pr_url=None, pr_revision=None, pr_num=None)